<a href="https://colab.research.google.com/github/flower-go/Prirodou-inspirovane-algoritmy/blob/master/du2_vysusilova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
#import tensorflow as tf
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.datasets import cifar10, mnist,  fashion_mnist
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.activations import linear, relu
#from tensorflow.nn import softmax

#import cleverhans.attacks

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.layers import Input

import numpy as np

from os import listdir
from os.path import isfile, join

import PIL
print('Pillow Version:', PIL.__version__)
from PIL import Image

In [ ]:
#nacteni ze zipu
from zipfile import ZipFile
from os.path import basename
import os
path_to_zip_file = "./dataset.zip"
with ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(".")

In [ ]:
#path = mypath + "/final/dataset/"
mypath = "./dataset/"
y_train = np.genfromtxt (mypath + "train/labels_train.csv", delimiter="\n", dtype=np.int32)
y_test = np.genfromtxt (mypath + "test/labels_test.csv", delimiter="\n", dtype=np.int32)

In [ ]:
def load_data(name):
    #path = mypath + "/final/dataset/" + name + "/"
    path = mypath + name + "/"
    x_images = [f for f in listdir(path) if isfile(join(path, f)) and f[0] != "." and f.split(".")[1] != "csv"]
    x = []
    for i in x_images:
        image = Image.open(path + i)
        image = np.asarray(image)
        x.append(image)
    x = np.asarray(x)
    return x

In [ ]:
x_train = load_data("train")
x_test = load_data("test")

In [ ]:
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

Model

In [ ]:
from keras.applications.vgg16 import VGG16
vgg_model = VGG16(weights="imagenet", include_top=False)
inputShape = (224,224)
preprocess = preprocess_input
from keras.models import Sequential
from keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
for layer in vgg_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False
num_classes = 5
model = Sequential()
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(vgg_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(.2))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
from keras.utils import to_categorical
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x=x_train, y=to_categorical(y_train), epochs=40, validation_data=(x_test, to_categorical(y_test)))

In [ ]:
model.fit(x=x_train, y=to_categorical(y_train), epochs=10, validation_data=(x_test, y_test))